## AlexNet from scratch model  

In [1]:
# importing libraries
import keras 
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import BatchNormalization
import numpy as np 
import pandas as pd

np.random.seed(1000)


# Instantiation
AlexNet = Sequential()

# 1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11),strides=(4,4),padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPool2D(pool_size=(2,2), strides=(2,2),padding='same'))

# 2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5,5),strides=(1,1),padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPool2D(pool_size=(2,2),padding='same'))

# 3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3),strides=(1,1),padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

# 4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3),strides=(1,1),padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

# 5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3),strides=(1,1),padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same'))


# Passing it to a Fully Connected layer
AlexNet.add(Flatten())

# 1st Fully connected Layer 
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent Overfitting 
AlexNet.add(Dropout(0.4))

# 2nd Fully connected Layer 
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout
AlexNet.add(Dropout(0.4))

# 3rd Fully connected Layer 
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout
AlexNet.add(Dropout(0.4))

# Output Layer
AlexNet.add(Dense(10))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

# Model Summary 
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 8, 96)          34944     
                                                                 
 batch_normalization (Batch  (None, 8, 8, 96)          384       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 8, 8, 96)          0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 4, 96)          0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 256)         614656    
                                                                 
 batch_normalization_1 (Bat  (None, 4, 4, 256)         1

In [2]:
# Compiling the model 
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [3]:
# Keras Library for CIFAR dataset
from keras.datasets import cifar10
(x_train, y_train),(x_test, y_test) = cifar10.load_data()

# Train-Validation-Test split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size =0.3)

# Dimension of the CIFAR10 dataset 
print((x_train.shape, y_train.shape))
print((x_val.shape, y_val.shape))
print((x_test.shape, y_test.shape))

((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))


In [4]:
# One-hot Encoding the Labels 
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

# Since we have 10 calsses we should expect the shape[1] of y_train, y_val and y_test to change from 1 to 10 
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)


# Verifying the dimension after one-hot encoding 
print((x_train.shape, y_train.shape))
print((x_val.shape, y_val.shape))
print((x_test.shape, y_test.shape))




((35000, 32, 32, 3), (35000, 10))
((15000, 32, 32, 3), (15000, 10))
((10000, 32, 32, 3), (10000, 10))


In [5]:
# Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=0.1)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=0.1)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=0.1)

# Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
# Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr = ReduceLROnPlateau( monitor='val_acc', factor= 0.01, patience= 3, min_lr= 1e-5)

In [7]:
# Defining the parameters
batch_size = 100
epochs = 10
learning_rate =0.001

In [8]:
# Training the Model 
AlexNet.fit(train_generator.flow(x_train, y_train, batch_size= batch_size),epochs=epochs, steps_per_epoch=x_train.shape[0]//batch_size, validation_data=val_generator.flow(x_val, y_val,batch_size= batch_size),validation_split=250,callbacks=[lrr], verbose=1)

Epoch 1/10
350/350 [==============================] - 561s 2s/step - loss: 1.6629 - accuracy: 0.4058 - val_loss: 2.2968 - val_accuracy: 0.1956 - lr: 0.0010
Epoch 2/10
350/350 [==============================] - 626s 2s/step - loss: 1.4088 - accuracy: 0.5031 - val_loss: 1.6885 - val_accuracy: 0.4138 - lr: 0.0010
Epoch 3/10
350/350 [==============================] - 551s 2s/step - loss: 1.2875 - accuracy: 0.5528 - val_loss: 2.1005 - val_accuracy: 0.2972 - lr: 0.0010
Epoch 4/10
350/350 [==============================] - 553s 2s/step - loss: 1.2016 - accuracy: 0.5819 - val_loss: 2.4567 - val_accuracy: 0.2349 - lr: 0.0010
Epoch 5/10
350/350 [==============================] - 551s 2s/step - loss: 1.1325 - accuracy: 0.6082 - val_loss: 1.5239 - val_accuracy: 0.4661 - lr: 0.0010
Epoch 6/10
350/350 [==============================] - 553s 2s/step - loss: 1.0628 - accuracy: 0.6332 - val_loss: 1.9500 - val_accuracy: 0.3836 - lr: 0.0010
Epoch 7/10
350/350 [==============================] - 550s 2s/st